# Reasoning Model Use Case Template

This template provides you a simple way to quickly build out production reasoning use cases.

Fill out each of the required sections with your own organization's data, from your use case.

A reference example from a banking credit use case has been included, as placeholder values.

In [1]:
from pydantic import BaseModel, Field
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
import json
import pandas as pd
from tools import execute_tool_function

# Load environment variables
load_dotenv("./.env")

client = AzureOpenAI(  
    api_version="2024-12-01-preview",  
    azure_endpoint=os.getenv("O3MINI_AZURE_ENDPOINT"),  
    api_key=os.getenv("O3MINI_API_KEY")  
)  


def o3minicall(prompt, reasoning_effort, tools=None, tool_choice="auto", response_format=None):
    """
    Call the O3 Mini model with optional tool calling functionality.
    
    Args:
        prompt (str): The user's query or prompt
        reasoning_effort (str): The reasoning effort level (low, medium, high)
        tools (list, optional): List of tool definitions for function calling
        tool_choice (str or dict, optional): Tool selection strategy
        response_format (dict, optional): Structured output format
        
    Returns:
        dict or str: The model's response, potentially including tool results
    """
    system_message = """
    You are a helpful AI assistant that analyzes credit risk based on historical data.
    """
    
    # Create the base parameters for the API call
    params = {
        "model": "o3-mini",
        "messages": [
            {
                "role": "user", 
                "content": system_message + " " + prompt
            }
        ],
        "reasoning_effort": reasoning_effort
    }
    
    # Add tools if provided
    if tools is not None:
        params["tools"] = tools
        params["tool_choice"] = tool_choice
    
    # Make the initial API call
    response = client.chat.completions.create(**params)
    response_message = response.choices[0].message
    
    # Check if the model wants to call a function
    if hasattr(response_message, 'tool_calls') and response_message.tool_calls:
        # Process function calls and get results
        tool_results = []
        
        for tool_call in response_message.tool_calls:
            # Extract function call details
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            # Execute the function
            result = execute_tool_function(function_name, function_args)
            
            tool_results.append({
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": json.dumps(result)
            })
        
        # Format the datasets from tool calls
        datasets_from_tool_calls = ""
        for result in tool_results:
            datasets_from_tool_calls += f"\n### Results from {result['name']}:\n"
            datasets_from_tool_calls += f"```json\n{result['content']}\n```\n"
        
        # Generate a new prompt with the tool call results incorporated
        new_prompt = f"""
        {prompt}
        
        Here is additional data retrieved to help with your analysis:
        {datasets_from_tool_calls}
        
        Please use this data to enhance your analysis.
        """
        
        # Create parameters for the second call with the new prompt
        second_params = {
            "model": "o3-mini",
            "messages": [
                {
                    "role": "user", 
                    "content": system_message + " " + new_prompt
                }
            ],
            "reasoning_effort": reasoning_effort
        }
        
        # Add response_format if provided
        if response_format is not None:
            second_params["response_format"] = response_format
            completion = client.beta.chat.completions.parse(**second_params)
            if hasattr(completion.choices[0].message, 'parsed'):
                return completion.choices[0].message.parsed
            else:
                return completion.choices[0].message.content
        else:
            completion = client.chat.completions.create(**second_params)
            return completion.choices[0].message.content
    
    # Handle the case where no function call is made
    if response_format is not None:
        # Switch to using parse for structured output
        parse_params = params.copy()
        parse_params["response_format"] = response_format
        completion = client.beta.chat.completions.parse(**parse_params)
        if hasattr(completion.choices[0].message, 'parsed'):
            return completion.choices[0].message.parsed
        else:
            return completion.choices[0].message.content
    else:
        return response_message.content


## Define inputs

### Instructions and task (required)

The instructions for what the model needs to do, and the specific input task.

In [2]:
instructions_and_task="""

You have been given an applicant. I need you to assess the credit risk for the applicant using the composite risk model. I will then take your recommendations and make a final determination.

 Your tasks are:

1. Calculate the average savings-to-expense ratio
2. Analyze the employment history to determine job stability.
3. Evaluate credit behavior 
4. Incorporate income consistency 
5. Factor in financial discipline and customer engagement 
6. Adjust the assessment using family details

Based on your analysis, compute the Composite Risk Rating and provide a detailed recommendation on whether the applicant should be approved for conventional credit, conditional credit with additional terms, or declined credit. Include all intermediate calculations, evidence from the datasets, and a discussion on which factors most significantly contribute to the risk score.

---

### <format>

Please include a brief executive summary at the top of your response, highlighting your key findings and final recommendation. Provide detailed intermediate calculations, reference specific datasets as evidence, and present your final composite risk score along with a clear explanation of each step using markdown formatting. All working out must be shown, and any tables must be fully filled out.

---

"""

In [3]:
# Define optional variables here- if they are commented out in lower sections, they will simply be blank strings.
few_shot_examples="No few shot examples provided"
provided_rules_and_policies="No provided rules and policies"


### Datasets (required)

The datasets needed to solve the problem.

Add your own datasets for the model to reason over.

Here, the data is included directly into the prompt as static text. In a production application, you will want to dynamically pull each data table specific to the input task/target customer, etc. This could involve a SQL query or Azure AI Search query.

An example of how this can be done is shown in the optional Function Calling section.

In [4]:
datasets="""
Dataset 1: Monthly Income and Recurring Expenses
*This table is presented as a bar chart comparing income vs. expenses over the last 6 months.*

| Month       | Income ($) | Recurring Expenses ($) |
|-------------|------------|------------------------|
| April 2023  | 5,200      | 3,800                  |
| May 2023    | 5,100      | 3,700                  |
| June 2023   | 5,250      | 3,850                  |
| July 2023   | 5,300      | 3,900                  |
| August 2023 | 5,150      | 3,750                  |
| September 2023 | 5,200   | 3,800                  |

---

#### Dataset 2: Employment History
*This dataset is represented as a timeline plot showing periods of employment at each company.*

| Employer             | Job Title                | Start Date  | End Date    | Years with Employer |
|----------------------|--------------------------|-------------|-------------|---------------------|
| AlphaTech Solutions  | Junior Analyst           | 2018-03-01  | 2020-08-31  | 2.5                 |
| Beta Innovations     | Senior Analyst           | 2020-09-15  | 2022-12-31  | 2.3                 |
| Gamma Financials     | Lead Financial Analyst   | 2023-01-15  | Present     | 0.8+                |

---

#### Dataset 3: Savings History
*This dataset is represented as a line graph tracking monthly savings deposits and ending balances.*

| Month       | Savings Deposited ($) | End-of-Month Savings Balance ($) |
|-------------|-----------------------|----------------------------------|
| April 2023  | 1,200                 | 7,500                            |
| May 2023    | 1,100                 | 8,600                            |
| June 2023   | 1,150                 | 9,750                            |
| July 2023   | 1,300                 | 11,050                           |
| August 2023 | 1,250                 | 12,300                           |
| September 2023 | 1,200              | 13,500                           |

---

#### Dataset 4: Detailed Spending Behavior
*This detailed dataset is presented as an interactive table. It includes 35 rows of transactions showing items purchased, amount spent, and transaction date. Note: The specific spending categories are not indicated to challenge identification of risk patterns.*

| Date       | Item Purchased                 | Amount ($) | Notes                            |
|------------|--------------------------------|------------|----------------------------------|
| 2023-09-01 | Coffee Shop Visit              | 5.50       |                                  |
| 2023-09-02 | Online Streaming Subscription  | 12.99      |                                  |
| 2023-09-03 | Grocery Store Purchase         | 65.20      |                                  |
| 2023-09-04 | Dinner at Restaurant           | 45.00      |                                  |
| 2023-09-05 | Book Purchase                  | 15.75      |                                  |
| 2023-09-06 | Gas Station                    | 30.00      |                                  |
| 2023-09-07 | Lottery Ticket                 | 3.00       |                                  |
| 2023-09-08 | Fast Food Meal                 | 10.50      |                                  |
| 2023-09-09 | Clothing Store                 | 80.00      |                                  |
| 2023-09-10 | Electronics Purchase           | 150.00     |                                  |
| 2023-09-11 | Casino Entry Fee               | 40.00      | Gambling-related expense         |
| 2023-09-12 | Grocery Store Purchase         | 70.30      |                                  |
| 2023-09-13 | Lottery Ticket                 | 5.00       |                                  |
| 2023-09-14 | Online Gaming Credit           | 25.00      |                                  |
| 2023-09-15 | Dinner at Restaurant           | 50.00      |                                  |
| 2023-09-16 | Coffee Shop Visit              | 6.00       |                                  |
| 2023-09-17 | Book Purchase                  | 18.00      |                                  |
| 2023-09-18 | Grocery Store Purchase         | 68.00      |                                  |
| 2023-09-19 | Fast Food Meal                 | 11.00      |                                  |
| 2023-09-20 | Cinema Ticket                  | 15.00      |                                  |
| 2023-09-21 | Online Streaming Subscription  | 12.99      |                                  |
| 2023-09-22 | Dining Out                     | 55.00      |                                  |
| 2023-09-23 | Lottery Ticket                 | 4.00       |                                  |
| 2023-09-24 | Gas Station                    | 32.00      |                                  |
| 2023-09-25 | Electronics Purchase           | 200.00     |                                  |
| 2023-09-26 | Fast Food Meal                 | 9.50       |                                  |
| 2023-09-27 | Coffee Shop Visit              | 5.25       |                                  |
| 2023-09-28 | Casino Table Fee               | 60.00      | Gambling-related expense         |
| 2023-09-29 | Grocery Store Purchase         | 75.00      |                                  |
| 2023-09-30 | Clothing Store                 | 90.00      |                                  |
| 2023-10-01 | Online Subscription Renewal    | 13.99      |                                  |
| 2023-10-02 | Lottery Ticket                 | 3.50       |                                  |
| 2023-10-03 | Dinner at Restaurant           | 48.00      |                                  |
| 2023-10-04 | Coffee Shop Visit              | 5.00       |                                  |
| 2023-10-05 | Fast Food Meal                 | 10.00      |                                  |

---

#### Dataset 5: Loan Repayment History
*This dataset is presented as a summary table with a line graph overlay indicating on-time performance trends.*

| Loan ID | Loan Amount ($) | Repayment Period (months) | On-Time Payment (Yes/No) | Default Occurrence |
|---------|-----------------|---------------------------|--------------------------|--------------------|
| L001    | 8,000           | 24                        | Yes                      | No                 |
| L002    | 12,000          | 36                        | Yes                      | No                 |
| L003    | 5,000           | 12                        | No                       | No                 |
| L004    | 15,000          | 48                        | Yes                      | No                 |
| L005    | 10,000          | 24                        | Yes                      | No                 |

---

#### Dataset 6: Credit Card Payment Timeliness
*This dataset is presented as a simple table and also visualized as a pie chart of on-time vs. late payments over a 6 month period.*

| Month       | On-Time Payment (Yes/No) |
|-------------|--------------------------|
| April 2023  | Yes                      |
| May 2023    | Yes                      |
| June 2023   | No                       |
| July 2023   | Yes                      |
| August 2023 | Yes                      |
| September 2023 | Yes                   |

---

#### Dataset 7: Customer Engagement and Feedback
*This dataset includes unstructured data from customer service interactions and survey comments.*

**Interaction 1:**
> Agent: Hello, thanks for contacting our support team. How can we help today?
>
> Customer: I appreciate the automated savings advice. I set up recurring transfers last month and it’s helped a lot.

**Interaction 2:**
> Email Feedback (2023-08-15): "I feel well supported by the bank’s financial guidance. My experience has generally been positive, though sometimes unclear about fees."

---

#### Dataset 8: Family Details
*This dataset is represented as a detailed table showing marital status, spouse income, dependents, and children ages. This data is essential in assessing overall financial resilience.*

| Field                | Details                                     |
|----------------------|---------------------------------------------|
| Marital Status       | Married                                     |
| Spouse Income ($/month) | 3,500                                  |
| Number of Dependents | 2                                           |
| Dependents' Ages     | 8, 14                                      |
| Additional Notes     | Family has moderate monthly expenditures for schooling and healthcare. |

---



"""

### Few shot examples (Optional)

This is optional- if you don't have time to create few shot examples, **you can simply comment out the cell below**. The use case will generally work well without it- this just provides a clear structure for the model to follow, allowing you to guide the format of the response.

The rules should cover all the possible scenarios you have- the few shot examples should just cover a handful, providing the **formatting** of the response, rather than covering all the **scenarios**. 1-10 few shot examples is generally used, 3 is generally typical.

In each few shot example, include the input, included datasets for solving the few shot example, and the output.

You can print out the structured object as a json using code like the below, to create the output:
json_output = result.model_dump_json(indent=2)
print(json_output)



In [5]:
few_shot_examples="""

<example_1>
<datasets>  
Dataset 1: Monthly Income and Recurring Expenses
 


| Month          | Income ($) | Recurring Expenses ($) |  
|----------------|------------|------------------------|  
| April 2023     | 3,200      | 3,500                  |  
| May 2023       | 3,100      | 3,600                  |  
| June 2023      | 3,250      | 3,700                  |  
| July 2023      | 3,200      | 3,800                  |  
| August 2023    | 3,150      | 3,750                  |  
| September 2023 | 3,200      | 3,850                  |  
 
 

Dataset 2: Employment History
 


| Employer             | Job Title                | Start Date  | End Date    | Years with Employer |  
|----------------------|--------------------------|-------------|-------------|---------------------|  
| AlphaTech Solutions  | Junior Analyst           | 2018-03-01  | 2019-08-31  | 1.4                 |  
| Beta Innovations     | Analyst                  | 2019-09-15  | 2021-06-30  | 1.8                 |  
| Freelance Consultant | Independent Contractor   | 2021-07-01  | Present     | 2.3                 |  
 
 

Dataset 3: Savings History
 


| Month          | Savings Deposited ($) | End-of-Month Savings Balance ($) |  
|----------------|-----------------------|----------------------------------|  
| April 2023     | 200                   | 1,000                            |  
| May 2023       | 150                   | 1,150                            |  
| June 2023      | 100                   | 1,200                            |  
| July 2023      | 50                    | 1,250                            |  
| August 2023    | 100                   | 1,350                            |  
| September 2023 | 150                   | 1,500                            |  
 
 

Dataset 4: Detailed Spending Behavior
 


| Date       | Item Purchased                 | Amount ($) | Notes                            |  
|------------|--------------------------------|------------|----------------------------------|  
| 2023-09-01 | Coffee Shop Visit              | 5.50       |                                  |  
| 2023-09-02 | Online Streaming Subscription  | 12.99      |                                  |  
| 2023-09-03 | Grocery Store Purchase         | 85.20      |                                  |  
| 2023-09-04 | Dinner at Restaurant           | 65.00      |                                  |  
| 2023-09-05 | Book Purchase                  | 25.75      |                                  |  
| 2023-09-06 | Gas Station                    | 40.00      |                                  |  
| 2023-09-07 | Lottery Ticket                 | 10.00      |                                  |  
| 2023-09-08 | Fast Food Meal                 | 15.50      |                                  |  
| 2023-09-09 | Clothing Store                 | 180.00     |                                  |  
| 2023-09-10 | Electronics Purchase           | 250.00     |                                  |  
| 2023-09-11 | Casino Entry Fee               | 60.00      | Gambling-related expense         |  
| 2023-09-12 | Grocery Store Purchase         | 90.30      |                                  |  
| 2023-09-13 | Lottery Ticket                 | 15.00      |                                  |  
| 2023-09-14 | Online Gaming Credit           | 50.00      |                                  |  
| 2023-09-15 | Dinner at Restaurant           | 75.00      |                                  |  
| 2023-09-16 | Coffee Shop Visit              | 6.00       |                                  |  
| 2023-09-17 | Book Purchase                  | 28.00      |                                  |  
| 2023-09-18 | Grocery Store Purchase         | 88.00      |                                  |  
| 2023-09-19 | Fast Food Meal                 | 20.00      |                                  |  
| 2023-09-20 | Cinema Ticket                  | 25.00      |                                  |  
| 2023-09-21 | Online Streaming Subscription  | 12.99      |                                  |  
| 2023-09-22 | Dining Out                     | 85.00      |                                  |  
| 2023-09-23 | Lottery Ticket                 | 14.00      |                                  |  
| 2023-09-24 | Gas Station                    | 42.00      |                                  |  
| 2023-09-25 | Electronics Purchase           | 300.00     |                                  |  
| 2023-09-26 | Fast Food Meal                 | 19.50      |                                  |  
| 2023-09-27 | Coffee Shop Visit              | 5.25       |                                  |  
| 2023-09-28 | Casino Table Fee               | 80.00      | Gambling-related expense         |  
| 2023-09-29 | Grocery Store Purchase         | 95.00      |                                  |  
| 2023-09-30 | Clothing Store                 | 190.00     |                                  |  
| 2023-10-01 | Online Subscription Renewal    | 13.99      |                                  |  
| 2023-10-02 | Lottery Ticket                 | 13.50      |                                  |  
| 2023-10-03 | Dinner at Restaurant           | 78.00      |                                  |  
| 2023-10-04 | Coffee Shop Visit              | 5.00       |                                  |  
| 2023-10-05 | Fast Food Meal                 | 20.00      |                                  |  
 
 

Dataset 5: Loan Repayment History
 


| Loan ID | Loan Amount ($) | Repayment Period (months) | On-Time Payment (Yes/No) | Default Occurrence |  
|---------|-----------------|---------------------------|--------------------------|--------------------|  
| L001    | 8,000           | 24                        | No                       | Yes                |  
| L002    | 12,000          | 36                        | No                       | Yes                |  
| L003    | 5,000           | 12                        | No                       | Yes                |  
| L004    | 15,000          | 48                        | No                       | Yes                |  
| L005    | 10,000          | 24                        | No                       | Yes                |  
 
 

Dataset 6: Credit Card Payment Timeliness
 


| Month          | On-Time Payment (Yes/No) |  
|----------------|--------------------------|  
| April 2023     | No                       |  
| May 2023       | No                       |  
| June 2023      | No                       |  
| July 2023      | No                       |  
| August 2023    | No                       |  
| September 2023 | No                       |  
 
 

Dataset 7: Customer Engagement and Feedback
 


**Interaction 1:**  
> Agent: Hello, thanks for contacting our support team. How can we help today?  
> Customer: I’m frustrated with the fees and lack of clarity in your services. I’m considering switching banks.  
  
**Interaction 2:**  
> Email Feedback (2023-08-15): "I feel unsupported by the bank’s financial guidance. My experience has been negative, especially with unexpected fees."  
 
 

Dataset 8: Family Details
 


| Field                | Details                                     |  
|----------------------|---------------------------------------------|  
| Marital Status       | Single                                      |  
| Spouse Income ($/month) | N/A                                      |  
| Number of Dependents | 3                                           |  
| Dependents' Ages     | 5, 10, 15                                   |  
| Additional Notes     | Family has high monthly expenditures for schooling and healthcare. |  
</datasets>

<instructions_and_task>
{instructions_and_task}
</instructions_and_task>

<function_call_results>
Year,Bankruptcy Status
2015,No
2016,No
2017,No
2018,No
2019,No
2020,No
2021,No
2022,No
2023,No
2024,No
2025,No
</function_call_results>

<output_json>
{
  "citations": [
    {
      "dataset_citation": "Dataset 1 & Dataset 3: Savings deposits and recurring expenses data were used to compute an average savings-to-expense ratio of approximately 3.4% (125/3700).",
      "reason_for_citation": "This calculation provided the basis for the low score assigned to Savings Behavior in the composite risk model."
    },
    {
      "dataset_citation": "Dataset 2: Employment history details showing a current tenure of 2.3 years in the latest role.",
      "reason_for_citation": "The current role being less than 3 years was used to assign a low score for Employment Stability."
    },
    {
      "dataset_citation": "Dataset 5 & Dataset 6: Loan repayment history and credit card payment timeliness consistently marked as 'No'.",
      "reason_for_citation": "These patterns underpin the low Credit Behavior score."
    },
    {
      "dataset_citation": "Dataset 1: Consistent monthly income values around $3200 with minimal fluctuations.",
      "reason_for_citation": "Indicated strong Income Consistency, meriting a high score."
    },
    {
      "dataset_citation": "Dataset 4 & Dataset 7: Spending behavior includes discretionary and gambling-related purchases; customer feedback shows significant dissatisfaction.",
      "reason_for_citation": "These were used to assign moderate Financial Discipline and low Engagement scores respectively."
    },
    {
      "dataset_citation": "Dataset 8: Family details showing three dependents with high extra expenditures for schooling and healthcare.",
      "reason_for_citation": "This information was used to adjust the overall risk rating downward given the added financial strain."
    },
    {
      "dataset_citation": "Bankruptcy data from get_bankruptcy_data indicating no bankruptcies until 2024 and 2025 being marked as 'Yes'.",
      "reason_for_citation": "This future projection of adverse events risk further supported lowering the composite risk score."
    }
  ],
  "response_to_input_task": "Executive Summary:\nThe applicant displays significant risk indicators in the composite risk analysis. Although the monthly income is consistent, the applicant presents a very low savings-to-expense ratio (approximately 3.4%), unstable employment (current role under 3 years), and poor credit behavior as evidenced by repeated missed on-time payments. The spending behavior shows discretionary and gambling-related transactions, and customer engagement feedback is negative. Moreover, family responsibilities with three dependents introduce additional financial stress, and recent bankruptcy trends in upcoming years add further concern. \n\nDetailed Analysis and Calculations:\n1. Savings-to-Expense Ratio Calculation:\n   - Total Savings Deposits over 6 months: $200 + $150 + $100 + $50 + $100 + $150 = $750\n   - Average Savings Deposits: $750 / 6 = $125\n   - Total Recurring Expenses over 6 months: $3,500 + $3,600 + $3,700 + $3,800 + $3,750 + $3,850 = $22,200\n   - Average Recurring Expenses: $22,200 / 6 ≈ $3,700\n   - Savings-to-Expense Ratio: 125 / 3700 ≈ 3.4%\n   - According to the scoring rules, a ratio below 30% results in a low score; thus, a score of approximately 40 is applied for Savings Behavior.\n\n2. Employment Stability:\n   - The applicant’s current tenure as a Freelance Consultant is 2.3 years (<3 years threshold), which results in a low score. A score of 45 is assigned for Employment Stability.\n\n3. Credit Behavior:\n   - Loan Repayment History (Dataset 5) shows that all payments were not on time and have resulted in defaults.\n   - Credit Card Payment Data (Dataset 6) indicates all months as 'No' for on-time payments.\n   - Consequently, a low score of around 40 is given for Credit Behavior.\n\n4. Income Consistency:\n   - Dataset 1 shows stable monthly income around $3200 with minimal variation. This consistency merits a high score; a score of 90 is assigned.\n\n5. Financial Discipline and Customer Engagement:\n   - Financial Discipline is moderate due to discretionary spending and gambling-related expenses observed in Dataset 4, resulting in a score of 50.\n   - Customer Engagement is low, based on negative feedback in Dataset 7, which justifies a score of 40.\n\n6. Family Details Impact:\n   - The additional financial burdens reflected by three dependents and high monthly expenditures for schooling and healthcare (Dataset 8) further increase the overall risk, warranting a downward adjustment of the final composite score.\n\nComposite Risk Score Calculation (using the Advanced Composite Risk Score Matrix):\n   • Savings Behavior (25% weight): 0.25 × 40 = 10\n   • Credit Behavior (25% weight): 0.25 × 40 = 10\n   • Employment Stability (15% weight): 0.15 × 45 = 6.75\n   • Income Consistency (15% weight): 0.15 × 90 = 13.5\n   • Financial Discipline (10% weight): 0.10 × 50 = 5\n   • Engagement and Feedback (10% weight): 0.10 × 40 = 4\n   Total = 10 + 10 + 6.75 + 13.5 + 5 + 4 = 49.25\n\nAdjustment Factors:\n   • Family Details: The presence of three dependents and high non-discretionary family expenditures reduce financial flexibility, prompting a further downward adjustment of approximately 4-5 points.\n   • Bankruptcy Data: Indications of potential bankruptcies in upcoming years (2024 and 2025) add further risk. \n\nFinal Adjusted Composite Risk Score ≈ 45 (rounded).\n\nRecommendation:\nBased on the composite risk score of approximately 45, the applicant falls into a high-risk category. Therefore, it is recommended that the applicant be offered conditional credit with additional terms. Such terms should include stricter monitoring, possible collateral requirements, and perhaps a review period to ensure improved financial discipline and stability. This approach balances the opportunity for credit access while mitigating potential risks due to financial instability and future adverse indicators.",
  "justification": "Every component of the analysis was derived by examining the relevant datasets provided. The very low savings-to-expense ratio, unstable employment, and poor credit behavior critically lowered the scores in those categories. While income consistency was a redeeming factor, it was offset by negative financial discipline feedback and pressing family financial obligations, as well as concerning future bankruptcy trends. These quantitative measures, combined with qualitative family data, informed the final composite risk score and the recommendation for conditional credit rather than outright approval.",
  "flag_for_human_review": false
}
</output_json>
</example_1>

<example_2>
<datasets>  
Dataset 1: Monthly Income and Recurring Expenses
 


| Month          | Income ($) | Recurring Expenses ($) |  
|----------------|------------|------------------------|  
| April 2023     | 6,500      | 3,200                  |  
| May 2023       | 6,700      | 3,300                  |  
| June 2023      | 6,800      | 3,250                  |  
| July 2023      | 6,750      | 3,400                  |  
| August 2023    | 6,900      | 3,300                  |  
| September 2023 | 7,000      | 3,350                  |  
 
 

Dataset 2: Employment History
 


| Employer             | Job Title                | Start Date  | End Date    | Years with Employer |  
|----------------------|--------------------------|-------------|-------------|---------------------|  
| AlphaTech Solutions  | Junior Analyst           | 2015-01-01  | 2018-12-31  | 4.0                 |  
| Beta Innovations     | Senior Analyst           | 2019-01-01  | 2021-12-31  | 3.0                 |  
| Gamma Financials     | Lead Financial Analyst   | 2022-01-01  | Present     | 1.8+                |  
 
 

Dataset 3: Savings History
 


| Month          | Savings Deposited ($) | End-of-Month Savings Balance ($) |  
|----------------|-----------------------|----------------------------------|  
| April 2023     | 1,500                 | 15,000                           |  
| May 2023       | 1,600                 | 16,600                           |  
| June 2023      | 1,700                 | 18,300                           |  
| July 2023      | 1,800                 | 20,100                           |  
| August 2023    | 1,900                 | 22,000                           |  
| September 2023 | 2,000                 | 24,000                           |  
 
 

Dataset 4: Detailed Spending Behavior
 


| Date       | Item Purchased                 | Amount ($) | Notes                            |  
|------------|--------------------------------|------------|----------------------------------|  
| 2023-09-01 | Coffee Shop Visit              | 4.50       |                                  |  
| 2023-09-02 | Online Streaming Subscription  | 12.99      |                                  |  
| 2023-09-03 | Grocery Store Purchase         | 60.20      |                                  |  
| 2023-09-04 | Dinner at Restaurant           | 40.00      |                                  |  
| 2023-09-05 | Book Purchase                  | 15.75      |                                  |  
| 2023-09-06 | Gas Station                    | 30.00      |                                  |  
| 2023-09-07 | Lottery Ticket                 | 2.00       |                                  |  
| 2023-09-08 | Fast Food Meal                 | 9.50       |                                  |  
| 2023-09-09 | Clothing Store                 | 70.00      |                                  |  
| 2023-09-10 | Electronics Purchase           | 120.00     |                                  |  
| 2023-09-11 | Grocery Store Purchase         | 65.30      |                                  |  
| 2023-09-12 | Online Gaming Credit           | 20.00      |                                  |  
| 2023-09-13 | Dinner at Restaurant           | 45.00      |                                  |  
| 2023-09-14 | Coffee Shop Visit              | 5.00       |                                  |  
| 2023-09-15 | Book Purchase                  | 18.00      |                                  |  
| 2023-09-16 | Grocery Store Purchase         | 70.00      |                                  |  
| 2023-09-17 | Fast Food Meal                 | 10.00      |                                  |  
| 2023-09-18 | Cinema Ticket                  | 12.00      |                                  |  
| 2023-09-19 | Online Streaming Subscription  | 12.99      |                                  |  
| 2023-09-20 | Dining Out                     | 50.00      |                                  |  
| 2023-09-21 | Gas Station                    | 28.00      |                                  |  
| 2023-09-22 | Electronics Purchase           | 150.00     |                                  |  
| 2023-09-23 | Fast Food Meal                 | 8.50       |                                  |  
| 2023-09-24 | Coffee Shop Visit              | 4.25       |                                  |  
| 2023-09-25 | Grocery Store Purchase         | 75.00      |                                  |  
| 2023-09-26 | Clothing Store                 | 85.00      |                                  |  
| 2023-09-27 | Online Subscription Renewal    | 13.99      |                                  |  
| 2023-09-28 | Dinner at Restaurant           | 48.00      |                                  |  
| 2023-09-29 | Coffee Shop Visit              | 5.00       |                                  |  
| 2023-09-30 | Fast Food Meal                 | 9.00       |                                  |  
| 2023-10-01 | Grocery Store Purchase         | 80.00      |                                  |  
| 2023-10-02 | Book Purchase                  | 20.00      |                                  |  
| 2023-10-03 | Dinner at Restaurant           | 50.00      |                                  |  
| 2023-10-04 | Coffee Shop Visit              | 4.50       |                                  |  
| 2023-10-05 | Fast Food Meal                 | 10.00      |                                  |  
 
 

Dataset 5: Loan Repayment History
 


| Loan ID | Loan Amount ($) | Repayment Period (months) | On-Time Payment (Yes/No) | Default Occurrence |  
|---------|-----------------|---------------------------|--------------------------|--------------------|  
| L001    | 8,000           | 24                        | Yes                      | No                 |  
| L002    | 12,000          | 36                        | Yes                      | No                 |  
| L003    | 5,000           | 12                        | Yes                      | No                 |  
| L004    | 15,000          | 48                        | Yes                      | No                 |  
| L005    | 10,000          | 24                        | Yes                      | No                 |  
 
 

Dataset 6: Credit Card Payment Timeliness
 


| Month          | On-Time Payment (Yes/No) |  
|----------------|--------------------------|  
| April 2023     | Yes                      |  
| May 2023       | Yes                      |  
| June 2023      | Yes                      |  
| July 2023      | Yes                      |  
| August 2023    | Yes                      |  
| September 2023 | Yes                      |  
 
 

Dataset 7: Customer Engagement and Feedback
 


**Interaction 1:**  
> Agent: Hello, thanks for contacting our support team. How can we help today?  
> Customer: I appreciate the automated savings advice. I set up recurring transfers last month and it’s helped a lot.  
  
**Interaction 2:**  
> Email Feedback (2023-08-15): "I feel well supported by the bank’s financial guidance. My experience has generally been positive, though sometimes unclear about fees."  
 
 

Dataset 8: Family Details
 


| Field                | Details                                     |  
|----------------------|---------------------------------------------|  
| Marital Status       | Married                                     |  
| Spouse Income ($/month) | 4,000                                  |  
| Number of Dependents | 2                                           |  
| Dependents' Ages     | 10, 12                                      |  
| Additional Notes     | Family has moderate monthly expenditures for schooling and healthcare. |  

</datasets>

<instructions_and_task>
{instructions_and_task}
</instructions_and_task>

<function_call_results>
Year,Bankruptcy Status
2015,No
2016,No
2017,No
2018,No
2019,No
2020,No
2021,No
2022,No
2023,No
2024,No
2025,No
</function_call_results>

<output_json>
{
  "citations": [
    {
      "dataset_citation": "Dataset 1: Monthly Income and Recurring Expenses; Dataset 3: Savings History",
      "reason_for_citation": "These datasets provided the monthly income, recurring expenses, and savings deposits data used to calculate the savings-to-expense ratio."
    },
    {
      "dataset_citation": "Dataset 2: Employment History",
      "reason_for_citation": "This dataset was used to assess job stability based on duration at each employer."
    },
    {
      "dataset_citation": "Dataset 5: Loan Repayment History and Dataset 6: Credit Card Payment Timeliness",
      "reason_for_citation": "They provided evidence of the applicant's credit behavior through on‐time repayments and payment consistency."
    },
    {
      "dataset_citation": "Dataset 1: Monthly Income and Recurring Expenses",
      "reason_for_citation": "Income variability and consistency were evaluated using month‐to‐month income figures."
    },
    {
      "dataset_citation": "Dataset 4: Detailed Spending Behavior; Dataset 7: Customer Engagement and Feedback",
      "reason_for_citation": "These datasets provided insights into financial discipline and customer engagement, including feedback and recorded spending behavior."
    },
    {
      "dataset_citation": "Dataset 8: Family Details",
      "reason_for_citation": "Family details (spouse income, number of dependents, and extra household expenditure notes) were used to adjust the final risk assessment."
    },
    {
      "dataset_citation": "get_bankruptcy_data results",
      "reason_for_citation": "Bankruptcy data indicating no bankruptcy for 2023 was used to reinforce the credit behavior assessment."
    }
  ],
  "response_to_input_task": "Executive Summary:\nThe applicant displays strong credit behavior and income consistency; however, savings-to-expense ratio is slightly below an ideal target and the current employment stability is weak. Family obligations further add pressure on disposable income. The composite risk score, calculated at approximately 75 (on a 0-100 scale), places the applicant in a moderate risk category. Consequently, the recommendation is to offer conditional credit with additional risk-based terms (for example, adjusted interest or collateral requirements) to mitigate the employment instability and family financial obligations.\n\nDetailed Calculations and Analysis:\n1. Average Savings-to-Expense Ratio:\n   - Data from Dataset 1 (Recurring Expenses) and Dataset 3 (Savings Deposits):\n     • April 2023: 1500/3200 ≈ 46.9%\n     • May 2023: 1600/3300 ≈ 48.5%\n     • June 2023: 1700/3250 ≈ 52.3%\n     • July 2023: 1800/3400 ≈ 52.9%\n     • August 2023: 1900/3300 ≈ 57.6%\n     • September 2023: 2000/3350 ≈ 59.7%\n   - Average Ratio ≈ (46.9 + 48.5 + 52.3 + 52.9 + 57.6 + 59.7) / 6 ≈ 53.5%\n   - Scoring: Rule specifies that >50% corresponds to a 'high' score range (70-100). An approximate score of 80 is assigned given the average is in the higher tier but still short of the ideal target of 66%.\n\n2. Employment Stability (Dataset 2):\n   - Tenures: 4.0 years (AlphaTech), 3.0 years (Beta Innovations), and a current period of 1.8+ years (Gamma Financials).\n   - Since the current employment is under 3 years, stability is classified as low (<50). An approximate score of 45 is assigned.\n\n3. Credit Behavior:\n   - Loan repayment history (Dataset 5) shows all on-time payments and no defaults. Likewise, credit card payment behavior (Dataset 6) is consistently on time.\n   - Thus, a 'high' rating is warranted with an assigned score of around 95.\n\n4. Income Consistency (Dataset 1):\n   - Monthly income has been stable with a positive trend from $6,500 to $7,000 over the past six months.\n   - Given regularity and stability, a score of 95 is assigned.\n\n5. Financial Discipline and Customer Engagement:\n   - Analysis of detailed spending (Dataset 4) shows moderate discretionary spending and no erratic behavior. Customer engagement (Dataset 7) indicates a generally positive sentiment despite minor concerns.\n   - Scores of 75 for financial discipline and 80 for engagement are assigned.\n\n6. Adjustment for Family Details (Dataset 8):\n   - While the applicant is married with a spouse contributing $4,000/month, the presence of two dependents and additional household expenditures (schooling and healthcare) slightly reduce disposable income.\n   - This factor is used as a downward adjustment of approximately 5 points on the overall composite risk score.\n\nComposite Risk Score Calculation:\n   - Savings Behavior (25% weight): 80 * 0.25 = 20.0\n   - Credit Behavior (25% weight): 95 * 0.25 = 23.75\n   - Employment Stability (15% weight): 45 * 0.15 = 6.75\n   - Income Consistency (15% weight): 95 * 0.15 = 14.25\n   - Financial Discipline (10% weight): 75 * 0.10 = 7.5\n   - Engagement and Feedback (10% weight): 80 * 0.10 = 8.0\n   - Sum = 20 + 23.75 + 6.75 + 14.25 + 7.5 + 8.0 = 80.25\n   - Adjusting for family obligations yields: 80.25 - 5 ≈ 75.25\n\nFinal Recommendations:\n   - Composite Risk Score: ~75/100\n   - Given a moderate composite score, the risk level is neither excellent nor poor. However, the low employment stability and family financial commitments elevate concern. Therefore, the recommendation is to offer conditional credit with additional terms (e.g., slightly higher interest rates, requirements for additional collateral or tighter monitoring) rather than a full conventional credit approval.\n   - The bankruptcy data (get_bankruptcy_data) confirms no recent bankruptcy filings, which supports the strong credit behavior but does not mitigate employment stability concerns.\n\nFinal Decision: Conditional Credit with Additional Terms",
  "justification": "The comprehensive evaluation integrated multiple dimensions of risk as prescribed by the advanced composite risk score matrix. Each component was scored based on dataset evidence, with adjustments made for family financial obligations. Employment stability was the most significant negative factor, while strong credit behavior and income consistency were positives. The final composite risk score of approximately 75 supports the recommendation for conditional credit, balancing the potential risks with the applicant's demonstrable financial strengths.",
  "flag_for_human_review": false
}
</output_json>
</example_2>
"""

### Rules & Policies - (Optional)

If you have a set idea of what rules/policies your scenario would need to follow for your problem/opportunity/scenario, include them here.

If you don't know what the rules or policies would be, you can leave this blank (comment out the block below).

The rules and policies should be comprehensive, and cover all the scenarios your model is likely to run into.

In [6]:
provided_rules_and_policies="""
**Rule:** The savings-to-expense ratio should be approximately 65% of the risk rating, while credit history data should make up 35% of the final evaluation. The overall assessment must consider multiple dimensions using the composite risk score matrix.  
  
#### Advanced Composite Risk Score Matrix  
  
| Component              | Description                                                                                                  | Weight (%) |  
|------------------------|--------------------------------------------------------------------------------------------------------------|------------|  
| Savings Behavior       | Savings-to-expense ratio and savings growth trend. Score <30%: low (<50), 30%-50%: moderate (50-70), >50%: high (70-100). | 24         |  
| Credit Behavior        | Credit card payment timeliness and past loan repayment adherence. Score <70%: low (<50), 70%-90%: moderate (50-70), >90%: high (70-100). | 26         |  
| Employment Stability   | Duration and advancement in current employment. <3 years: low (<50), 3-5 years: moderate (50-70), >5 years: high (70-100).  | 16         |  
| Income Consistency     | Regularity and stability of monthly income. Consistency above 90% receives high score.                        | 14         |  
| Financial Discipline   | Evidence of proactive financial management from customer interactions.                                      | 10         |  
| Engagement and Feedback| Customer engagement with bank services and satisfaction survey scores. High (average >8), moderate (7-8), low (<7). | 10         |  
  
**Composite Risk Rating Calculation:**  
Composite Risk Rating = Σ (Weight Factor × Assigned Score) (each component assigned a score 0-100).  
  
---  
  
**Rule:**"The model must take into account family details — including whether the spouse is earning, the number of dependents, and the ages of any children. These factors can significantly influence overall financial stability and risk."  
"""

### Structured Outputs (Optional)

You can guide o3-mini to provide a JSON output, with specific outputs. You can provide a description and data type, to help the model provide additional information needed to make decisions.

**Modifying the structured output is optional**- feel free to add in other fields you want the model to respond with to the object below! **Otherwise leave it as the default values.**

The current response object provides an example of a typical response.

Here is an example of what a structured output could look like for your use case, to better integrate with downstream APIs. For example, you could output the result of your use case in a JSON format ready for ingestion by downstream systems.

class Components(BaseModel):
    Operation_Number: str
    Part_Description: str

class MainWorkCenter(BaseModel):
    Main_Work_Center: str = Field(..., description="4-letter alphanumeric code")
    Work_Order: str = Field(..., description="Work Order number, always begins with 'WO'")
    Functional_Location: str = Field(..., description="Location where the resource is located")
    Components: Components

In [7]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Citation(BaseModel):
    dataset_citation: str = Field(..., description="The exact datapoint, text, or section of provided data that supports the claim made in the analysis.")
    reason_for_citation: str = Field(..., description="A reason for why this datapoint is relevant to the analysis.")

class Response(BaseModel):
    citations: list[Citation] = Field(..., 
        description="A list of citations of specific data points, text, or sections of provided data that supports the claim made in the analysis.")
    response_to_input_task: str = Field(..., 
        description="A comprehensive response to instruction or task that has been given..")
    justification: str = Field(..., 
        description="A comprehensive overview of the decisions made, the specific data points made, for the purpose of audit logging and tracking decisioning..")
    flag_for_human_review: bool = Field(..., 
        description="If the outcome of the analysis is not clear, or if the analysis is not conclusive, then flag the case study for human review.")


### Function Calling Code (Optional)

This is an example of how to add tools to your AI system. You could imagine we are retrieving historical data from a database using SQL.

In this case, to keep the repository simple, the function defined in tools.py is simply querying a locally stored csv file, but you can simple add your own code, and easily create any integration!

The AI will determine which tools (or none!) to use based on the description field, so ensure you provide a good, human interpretable description. The AI will also decide what arguments to pass to the function, ensure the parameter descriptions are clear. For example, here it can choose to either search for a specific year, or leave it blank in order to retrieve all the years of historical information.

The retrieved data is appended in a separate message after the prompt, and then the model runs the full analysis.


In [8]:
database_query_tool = [
    {
        "type": "function",
        "function": {
            "name": "get_bankruptcy_data",
            "description": "Retrieve bankruptcy data for specified years or all available years if no years are specified.",
            "parameters": {
                "type": "object",
                "properties": {
                    "years": {
                        "type": "array",
                        "items": {
                            "type": "integer"
                        },
                        "description": "List of years to retrieve bankruptcy data for. If empty or not provided, returns data for all available years."
                    }
                },
                "required": ["years"],  # Making years required to satisfy the API
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

## Construct the prompt for the use case
Here we take all the different inputs

In [9]:
prompt=f"""


You are helping process input data into an output format.

Here is background information and rules and policies to follow:
<provided_rules_and_policies>
{provided_rules_and_policies}
</provided_rules_and_policies>

Here are past examples of the input and output:
<few_shot_examples>
{few_shot_examples}
</few_shot_examples>


Here are the datasets:
<datasets>  
{datasets}
</datasets>

Here is the input instructions and task from the user:
<instructions_and_task>
{instructions_and_task}
</instructions_and_task>

"""

print(prompt)







You are helping process input data into an output format.

Here is background information and rules and policies to follow:
<provided_rules_and_policies>



**Rule:** The savings-to-expense ratio should be around 66% of the risk rating and credit history data should constitute 33% of the final evaluation. The overall assessment must consider multiple dimensions using the composite risk score matrix. 

#### Advanced Composite Risk Score Matrix

| Component             | Description                                                                                                  | Weight (%) |
|-----------------------|--------------------------------------------------------------------------------------------------------------|------------|
| Savings Behavior      | Savings-to-expense ratio and savings growth trend. Score <30%: low (<50), 30%-50%: moderate (50-70), >50%: high (70-100). | 25         |
| Credit Behavior       | Credit card payment timeliness and past loan repayment adher

## Generate a prediction


In [10]:
result = o3minicall(
        prompt=prompt,
        reasoning_effort="low",
        tools=database_query_tool,
        tool_choice="auto",
        response_format=Response
)
print(result)

Executing get_bankruptcy_data with years: []. If no year specified, all years will be returned.
citations=[Citation(dataset_citation='Dataset 1 and Dataset 3: Monthly Income, Recurring Expenses, and Savings History', reason_for_citation='These datasets were used to compute the savings-to-expense ratio. The calculation shows ratios ranging between about 29.7% and 33.3%, with an average of approximately 31.6%, which places the savings behavior in the moderate category.'), Citation(dataset_citation='Dataset 2: Employment History', reason_for_citation='The applicant’s employment spans three roles with tenures of 2.5 years, 2.3 years, and 0.8+ years. All three indicate an employment duration below the ideal thresholds and contribute to a low stability rating.'), Citation(dataset_citation='Dataset 5 and Dataset 6: Loan Repayment History and Credit Card Payment Timeliness', reason_for_citation='The credit history shows mostly on‐time loan repayments and credit card payments, with one instance

### Prediction Output

In [11]:
# See the model's response to the input task.
print(result.response_to_input_task)

Executive Summary:
The analysis of the applicant’s financial profile reveals a composite risk score in the moderate range of approximately 58–61 on a 0–100 scale, after adjustments. While the applicant shows strong income consistency and generally acceptable credit behavior, concerns regarding moderate savings behavior and limited employment stability, compounded by upcoming bankruptcy risks and family expenditures, suggest that the applicant should be offered conditional credit with additional terms rather than full conventional credit approval.

Detailed Calculations and Analysis:

1. Savings-to-Expense Ratio (Savings Behavior):
   - April: 1,200 / 3,800 = 31.6%
   - May: 1,100 / 3,700 = 29.7%
   - June: 1,150 / 3,850 = 29.9%
   - July: 1,300 / 3,900 = 33.3%
   - August: 1,250 / 3,750 = 33.3%
   - September: 1,200 / 3,800 = 31.6%
   → Average ≈ 31.6% 
   → According to our scoring guide (30%-50% is moderate), we assign a score of approximately 60 for Savings Behavior (Weight = 25%).


In [12]:
# See the explanation of why the model made the decision it did. Note this is an open field of research- for numerical calculations, with step by step reasoning that arrives at an answer, this is reliable.
print(result.justification)

Each component of the composite risk score was derived by analyzing the relevant datasets. The moderate savings-to-expense ratio, low employment tenure, and an instance of credit payment delays contribute negatively, while income consistency and positive customer engagement partially offset these weaknesses. Additional adjustments for family responsibilities and prospective bankruptcy risks further reduce the final composite score, leading to a recommendation of conditional credit with risk-mitigating terms.


In [13]:
# See the data points the model used to make the decision it did.
print(result.citations)

[Citation(dataset_citation='Dataset 1 and Dataset 3: Monthly Income, Recurring Expenses, and Savings History', reason_for_citation='These datasets were used to compute the savings-to-expense ratio. The calculation shows ratios ranging between about 29.7% and 33.3%, with an average of approximately 31.6%, which places the savings behavior in the moderate category.'), Citation(dataset_citation='Dataset 2: Employment History', reason_for_citation='The applicant’s employment spans three roles with tenures of 2.5 years, 2.3 years, and 0.8+ years. All three indicate an employment duration below the ideal thresholds and contribute to a low stability rating.'), Citation(dataset_citation='Dataset 5 and Dataset 6: Loan Repayment History and Credit Card Payment Timeliness', reason_for_citation='The credit history shows mostly on‐time loan repayments and credit card payments, with one instance of a delayed credit card payment and one loan with a ‘No’ on time record. This justifies assigning a mode